In [ ]:
import Pkg; Pkg.instantiate()
Pkg.activate(".")
using DrugResponseModel
using Plots
Plots.scalefontsizes(0.7)

concs, _, g1s1, g2s1 = load(189, 1);
g0 = g1s1[1,1,1]+g2s1[1,1,1];

# this is the optimum estimated parameters for all drugs at once, and considers % in G1 separately for drugs. ==> 41 parameters
p = DrugResponseModel.return_param41();


In [ ]:
function plt_heatmap_combin(drug1_ind, drug2_ind, effs, g0, concs, controld1, controld2)
    names = ["Lpt", "Dox", "Gem", "Tax", "Palb"]
    # bliss on data
    BlissData = zeros(8, 8)
    for i=1:8
        for j=1:8
            BlissData[i, j] = DrugResponseModel.calc_cellNum(effs[:, i, drug1_ind], effs[:, j, drug2_ind], g0, controld1, controld2)
        end
    end
    bliss_comb = DrugResponseModel.AllBliss_params(effs[:, :, drug1_ind], effs[:, :, drug2_ind])
    bliss_comb_cellnum = BlissModelComb(bliss_comb, g0)
    diff = bliss_comb_cellnum .- BlissData
    frac = bliss_comb_cellnum ./ BlissData

    p1=Heatmap(concs, bliss_comb_cellnum, drug1_ind, drug2_ind, names[drug1_ind], names[drug2_ind], "Bliss Cell Cycle Model")
    p2=Heatmap(concs, BlissData, drug1_ind, drug2_ind, names[drug1_ind], names[drug2_ind], "Bliss Data")
    p3=Heatmap(concs, diff, drug1_ind, drug2_ind, names[drug1_ind], names[drug2_ind], "model - data", clim_min = -15.0, clim_max = 15.0)
    p4=Heatmap(concs, frac, drug1_ind, drug2_ind, names[drug1_ind], names[drug2_ind], "model / data", clim_min = 0.01, clim_max = 2.5)

    return plot(p1, p2, p3, p4, layout=(2, 2))
end

In [ ]:
using Interact
function slider_Bliss(p, drug1_ind, drug2_ind, concs, indx1, indx2, g0, controld1, controld2)
    
    @manipulate for maxg1_prog=0.0:0.01:1.0, maxg2_prog=0.0:0.01:1.0, maxg1_death=0.0:0.01:1.0, maxg2_death=0.0:0.001:1.0, maxg1_prog2=0.0:0.01:1.0, maxg2_prog2=0.0:0.01:1.0, maxg1_death2=0.0:0.01:1.0, maxg2_death2=0.0:0.01:1.0

        p[indx1:indx1+3] .= [maxg1_prog, maxg2_prog, maxg1_death, maxg2_death]
        p[indx2:indx2+3] .= [maxg1_prog2, maxg2_prog2, maxg1_death2, maxg2_death2]
        
        effss = getODEparamsAll(p, concs)
        plt_heatmap_combin(drug1_ind, drug2_ind, effss, g0, concs, controld1, controld2)
    end
end

In [ ]:
# lapatinib
efcs = getODEparamsAll(p, concs)
t = LinRange(0.0, 95.0, 189)
controls = zeros(5)
for i=1:5
    g1, g2, _ = predict(efcs[:, 1, i], g0, t)
    controls[i] = g1[end] + g2[end]
end
DrugResponseModel.plot_parameters(concs[:, 1], efcs[:, :, 1], zeros(4,7))

In [ ]:
# palbociclib
DrugResponseModel.plot_parameters(concs[:, 5], efcs[:, :, 5], zeros(4,7))

In [ ]:
# g1 and g2 inhibit
slider_Bliss(p, 1, 5, concs, 3, 31, g0, controls[1], controls[5])

In [ ]:
### conclusion: 
#### decrease of progression rate has more effect in model than data

In [ ]:
# doxorubicin
DrugResponseModel.plot_parameters(concs[:, 2], efcs[:, :, 2], zeros(4,7))

In [ ]:
# gemcitabine
DrugResponseModel.plot_parameters(concs[:, 3], efcs[:, :, 3], zeros(4,7))

In [ ]:
# g1 and g2 cell death: doxorubicin and gemcitabine
slider_Bliss(p, 2, 3, concs, 10, 17, g0, controls[2], controls[3])

In [ ]:
### conclusion:
when two drugs have cell death effects, changing the progression rates won't make a significant difference.

In [ ]:
# g1 inhibit and both phases cell death: lapatinib and gemcitabine
slider_Bliss(p, 1, 3, concs, 3, 17, g0, controls[1], controls[3])

In [ ]:
# taxol parameters
DrugResponseModel.plot_parameters(concs[:, 4], efcs[:, :, 4], zeros(4,7))

In [ ]:
# both g1 inhibition: taxol and lapatinib
slider_Bliss(p, 1, 4, concs, 3, 24, g0, controls[1], controls[4])

In [ ]:
# both cell death: taxol and gemcitabine
slider_Bliss(p, 3, 4, concs, 17, 24, g0, controls[3], controls[4])

In [ ]:
### conclusion:
#### It seems that the conventional method says increasing the gemcitabine's concentration doesn't make any difference, but the model says it has a slight effect.